In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt

In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import seaborn as sns
sns.set()

In [5]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

In [6]:
from sklearn.model_selection import KFold, StratifiedKFold

In [10]:
train = pd.read_csv("../data/dacon_galaxy/train.csv", index_col=0)
test = pd.read_csv('../data/dacon_galaxy/test.csv', index_col=0)
sample_submission = pd.read_csv('../data/dacon_galaxy/sample_submission.csv', index_col=0)

In [11]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [12]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type_num
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340,8
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314,8
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928,8
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855,8
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655,10


In [13]:
train_X = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_X = test

In [14]:
train['type'].value_counts().sum()

199991

In [15]:
train['type'].value_counts()

QSO                    49680
GALAXY                 37347
SERENDIPITY_BLUE       21760
SPECTROPHOTO_STD       14630
REDDEN_STD             14618
STAR_RED_DWARF         13750
STAR_BHB               13500
SERENDIPITY_FIRST       7132
ROSAT_D                 6580
STAR_CATY_VAR           6506
SERENDIPITY_DISTANT     4654
STAR_CARBON             3257
SERENDIPITY_RED         2562
STAR_WHITE_DWARF        2160
STAR_SUB_DWARF          1154
STAR_BROWN_DWARF         500
SKY                      127
SERENDIPITY_MANUAL        61
STAR_PN                   13
Name: type, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(train_X, 
                                                    train_y, 
                                                    stratify = train_y,
                                                    test_size=0.2, 
                                                    random_state=1234)

### stratifiedKFold를 사용하자

In [17]:
#kfold = KFold(n_splits=6, random_state= 0, shuffle = True)
kfold_s = StratifiedKFold(n_splits=6, random_state= 0, shuffle = True)

# CatBoost

In [7]:
import catboost as catb
from catboost import CatBoostClassifier

In [23]:
CBC = CatBoostClassifier(random_state=1234)

cbc_param_grid = {
    #'n_estimators' : [10, 30, 50],
    'max_depth' : np.arange(3,10,1),
    'learning_rate' : np.arange(0.13,0.19,0.02),
    'iterations' : [1000],
#     'depth' : [8],
    'loss_function' : ['MultiClass'],
    'leaf_estimation_method' : ['Newton'],
    'eval_metric' : ['Accuracy']
}

In [24]:
n_iter_search = 10
CBC_rgs = RandomizedSearchCV(CBC, 
                             cbc_param_grid, 
                             cv=kfold_s,
                             random_state=1234,
                             scoring='neg_log_loss')

In [25]:
CBC_rgc = CBC_rgs.fit(X_train, y_train)

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


0:	learn: 0.7153540	total: 1.58s	remaining: 26m 20s
1:	learn: 0.1113486	total: 2.89s	remaining: 24m 3s
2:	learn: 0.6200195	total: 4.25s	remaining: 23m 33s
3:	learn: 0.6364836	total: 5.58s	remaining: 23m 10s
4:	learn: 0.6112361	total: 6.92s	remaining: 22m 57s
5:	learn: 0.5257426	total: 8.32s	remaining: 22m 59s
6:	learn: 0.5605011	total: 9.68s	remaining: 22m 52s
7:	learn: 0.5761251	total: 11.1s	remaining: 23m 1s
8:	learn: 0.6327858	total: 12.5s	remaining: 22m 58s
9:	learn: 0.6200645	total: 14.1s	remaining: 23m 12s
10:	learn: 0.6362061	total: 15.7s	remaining: 23m 28s
11:	learn: 0.6317132	total: 17.2s	remaining: 23m 33s
12:	learn: 0.6916967	total: 18.5s	remaining: 23m 27s
13:	learn: 0.7048005	total: 19.9s	remaining: 23m 22s
14:	learn: 0.6779703	total: 21.3s	remaining: 23m 21s
15:	learn: 0.6927993	total: 22.8s	remaining: 23m 23s
16:	learn: 0.7225848	total: 24.3s	remaining: 23m 24s
17:	learn: 0.6793804	total: 26s	remaining: 23m 39s
18:	learn: 0.6861761	total: 27.5s	remaining: 23m 41s
19:	lea

KeyboardInterrupt: 

In [28]:
CBC_rgc.best_estimator_.get_all_params()

{'nan_mode': 'Min',
 'gpu_ram_part': 0.95,
 'eval_metric': 'Accuracy',
 'iterations': 1000,
 'fold_permutation_block': 64,
 'leaf_estimation_method': 'Newton',
 'observations_to_bootstrap': 'TestOnly',
 'grow_policy': 'SymmetricTree',
 'boosting_type': 'Plain',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'devices': '0:1',
 'pinned_memory_bytes': '104857600',
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'fold_size_loss_normalization': False,
 'gpu_cat_features_storage': 'GpuRam',
 'use_best_model': False,
 'class_names': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18],
 'random_seed': 1234,
 'depth': 8,
 'has_time': False,
 'fold_len_multiplier': 2,
 'border_count': 128,
 'min_fold_size': 100,
 'data_partition': 'DocParallel',
 'bagging_temperature': 1,
 'classes_count': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 

In [29]:
CBC_rgc.best_score_

-0.37699149297582524

In [30]:
CBC_best = CBC_rgc.best_estimator_
y_pred = CBC_best.predict_proba(test_X)

In [31]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission.csv', index=True)